In [1]:
import json
import pandas as pd
from glob import glob
from tqdm import tqdm
tqdm.pandas()
import numpy as np
from scripts.graph_funcs import *
from scripts.network_graph import make_network
import networkx as nx


In [2]:
file_list=glob('dataset/*/*.json')
df=pd.DataFrame.from_dict(json.load(open(file_list[0])), orient="index").T
for patent_loc in file_list[1:]:
    patent_file=json.load(open(patent_loc))
    temp_df=pd.DataFrame.from_dict(patent_file, orient="index").T
    df=pd.concat([df, temp_df], ignore_index=True)

In [3]:
df['publication_date']= pd.to_datetime(df['publication_date'])
df['publication_year']=df['publication_date'].apply(lambda k: k.year)

In [4]:
# G=make_network(df, use_abstract=True, use_cpc=True, file_loc='dataset/network.adjlist')

100%|██████████| 271216/271216 [02:04<00:00, 2181.38it/s]


In [4]:
G=nx.read_adjlist('dataset/network.adjlist')

In [5]:
classificiation_patent={}

for idx in df.index:
    for cpc in df['classifications'][idx]:
        if cpc in classificiation_patent:
            classificiation_patent[cpc].append(df['patent_id'][idx])
        else:
            classificiation_patent[cpc]=[df['patent_id'][idx]]

In [6]:
patent_to_check='US-4658370-A'

In [7]:
starter_nodes=[]
for cpc in df[df['patent_id']==patent_to_check]['classifications'].values[0]:
    starter_nodes.append(np.random.choice(classificiation_patent[cpc]))

In [8]:
random_walks=[]
for node in starter_nodes:
    random_walks.append(random_walk(G, node))

In [9]:
from functools import reduce

common_patents = list(reduce(lambda i, j: i & j, (set(x) for x in random_walks)))

In [10]:
matched_nodes={}
if patent_to_check in common_patents:
    common_patents.remove(patent_to_check)
for patent in common_patents:
    matched_nodes[patent]=get_graph_similarity(patent_to_check, patent)

In [11]:
matched_nodes

{'US-11132609-B2': [['1', '21'],
  ['58', '21'],
  ['66', '1'],
  ['66', '21'],
  ['61', '1'],
  ['61', '21'],
  ['61', '22']],
 'US-11022719-B2': [['66', '2'],
  ['66', '16'],
  ['61', '15'],
  ['61', '2'],
  ['61', '16']],
 'US-11475330-B2': [['59', '1'], ['66', '1'], ['61', '1']],
 'US-11423886-B2': [['59', '19']],
 'US-11403526-B2': [],
 'US-11357502-B2': [],
 'US-11501194-B2': [['50', '1'], ['59', '11'], ['66', '11']],
 'US-11675943-B2': [['58', '1'], ['59', '1'], ['66', '1'], ['61', '1']],
 'US-9349100-B2': [],
 'US-2021232955-A1': [['59', '1'],
  ['59', '7'],
  ['59', '13'],
  ['59', '19'],
  ['59', '22'],
  ['66', '1'],
  ['66', '7'],
  ['66', '13'],
  ['66', '19'],
  ['66', '22'],
  ['60', '1'],
  ['60', '7'],
  ['60', '13'],
  ['61', '1'],
  ['61', '7'],
  ['61', '13'],
  ['61', '19'],
  ['61', '22'],
  ['63', '1'],
  ['63', '13']]}